In [2]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from torch import optim, nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from numpy import genfromtxt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb


import copy
%matplotlib inline

rng = 0

test_values = pd.read_csv("./test-set-values.csv")
train_values = pd.read_csv("./training-set-values.csv")
train_labels = pd.read_csv("./training-set-labels.csv")


In [3]:
print(train_values.columns)
print(train_values.shape)


Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')
(59400, 40)


In [30]:
print("any nulls in train_values?")
print(pd.isnull(train_values).any()[lambda x: x==True])
print()
print("any NANs in train_values?")
print(pd.isna(train_values).any()[lambda x:x==True])


any nulls in train_values?
funder               True
installer            True
subvillage           True
public_meeting       True
scheme_management    True
scheme_name          True
permit               True
dtype: bool

any NANs in train_values?
funder               True
installer            True
subvillage           True
public_meeting       True
scheme_management    True
scheme_name          True
permit               True
dtype: bool


PS: The features not shown are false for both cases

## train_values have the same features with both NANs and nulls.

In [6]:
train_values.iloc[:, :10].head()


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0


In [7]:
train_values.iloc[:, 11:20].head()


,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by
0,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd
1,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd
2,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd
3,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd
4,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd


In [8]:
train_values.iloc[:, 21:30].head()


,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment
0,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually
1,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay
2,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket
3,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay
4,NaN,True,0,gravity,gravity,gravity,other,other,never pay


In [9]:
train_values.iloc[:,31:40].head()

,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [10]:
print("any nulls or NANs in train_labels?")
print(pd.isnull(train_labels).any())
print(pd.isna(train_labels).any())


any nulls or NANs in train_labels?
id              False
status_group    False
dtype: bool
id              False
status_group    False
dtype: bool
